# Spark Recommender System
영화평점데이터를 활용한 추천시스템 개발

## 1. Data-Preprocessing
### 1.1 Loading Data

In [1]:
val rawData = sc.textFile("/home/paulkim/workspace/Spark/semi_project/data/ml-100k/u.data")
rawData.first()

196	242	3	881250949

### 1.2 EDA u.data
- **u.user schema**: user id, movie id, rating

In [2]:
val rawRatings = rawData.map(_.split("\t").take(3))
rawRatings.count()

100000

In [3]:
println("Number of users : " + rawRatings.map(arr => arr(0)).distinct().count())
println("Number of movies : " + rawRatings.map(arr => arr(1)).distinct().count())
rawRatings.first() // Array[String] = Array(196, 242, 3)

Number of users : 943
Number of movies : 1682


Array(196, 242, 3)

## 2. ALS recommendations model
**org.apache.spark.mllib.recommendation.ALS**
- ratings : org.apache.spark.mllib.recommendation.Rating
- rank : Int
- iterations : Int
- lambda : Double

**org.apache.spark.mllib.recommedation.Rating**
- user : Int
- prodict: Int
- rating : Double

### 2.1 Import Libs & Ratings Object

In [4]:
import org.apache.spark.mllib.recommendation.ALS
import org.apache.spark.mllib.recommendation.Rating

In [5]:
val ratings = rawRatings.map{ case Array(user, movie, rating) => Rating(user.toInt, movie.toInt, rating.toDouble)}
ratings.first() // Rating(196,242,3.0)

Rating(196,242,3.0)

### 2.2 Training Model
- **rank** : ALS모델에서 사용하는 특징 갯수. 낮은 순위의 근사치 행렬에서 숨겨진 특징 갯수임. 메모리에 직접적인 영향을 미침. 특징 갯수와 메모리 사용 간에는 트레이드오프임. 10에서 200 사이의 범위
- **iterations** : 실행에 대한 반복 횟수. 10회 정도를 권장 
- **lambda** : 오버피팅의 문제를 제어. penalty이며 값이 높을수록 regularization을 수행함.

In [6]:
// .recommendation.MatrixFactorizationModel@c938ed5
val model = ALS.train(ratings, 50, 10, 0.01)

ALS.train()은 (id, factor) 쌍으로 구성된 RDD 형태로 user와 item을 포함하는 MatrixFactorizationModel객체를 변환함.

In [7]:
model.userFeatures.count // users

943

In [8]:
model.productFeatures.count // items

1682

사용자 943명(User)과 1682개의 영화(Item)가 여전히 존재하고 있는 것을 확인할 수 있음

## 3. 사용자 추천(users에게 items(영화)를 추천)

**tip) 추천 방안 2가지**
- **사용자 기반의 방법** : item선택이 유사한 사용자가 부여한 점수는 특정 사용자에게 추천 
- **제품 기반의 방법** : 사용자가 후보 제품에 평점을 부여한 유사 제품을 바탕으로 추천 연산을 수행


### 3.1 predict model(평점예측) => 특별한 의미X
userId가 789인 사람이 itemId가 123인 영화의 평점을 예측

In [9]:
// 789번 사용자와 영화 123번에 대한 평점을 예측
val predictedRating = model.predict(789, 123) // predictedRating: Double = 3.2936442480956147

3.2936442480956147점. 약, 3.29점의 평점을 예측한 것을 확인할 수 있음

predict메서드는 입력으로 (user, item) 아이디로 구성된 RDD를 매개변수로 받아 각 ID에 대한 예측 값을 생성함.

### 3.2 사용자에게 10개의 item(영화)를 추천
특정 사용자에게 제공할 수 있는 top-K 추천 제품 목록을 생성할 때 MatrixFactorizationModel은 recommendProduct라는 메서드를 사용할 수 있음. 
- user : user id
- num : 추천 갯수

tip) recommendProducts는 예측 점수의 순서대로 정렬하여 결과를 반환. 예측 점수는 사용자 특징 벡터와 각 제품 특징 벡터 간의 내적으로 계산

In [10]:
val userId = 789
val K = 10
val topKRecs = model.recommendProducts(userId, K)
println(topKRecs.mkString("\n"))

Rating(789,12,5.913209374433712)
Rating(789,199,5.855880700816102)
Rating(789,526,5.825030838268482)
Rating(789,182,5.75696475422482)
Rating(789,96,5.6864516894478525)
Rating(789,250,5.684725442684111)
Rating(789,144,5.6774028331597926)
Rating(789,183,5.621820401359451)
Rating(789,56,5.614443663294212)
Rating(789,219,5.610997103454043)


### 3.3 추천 목록 확인작업
앞서 예측에 사용된 영화 제목 확인해봄

In [11]:
val movies = sc.textFile("/home/paulkim/workspace/Spark/Spark_ML/data/ml-100k/u.item")
val titles = movies.map(line => line.split("\\|").take(2)).map(array => (array(0).toInt, array(1))).collectAsMap()
titles(123)

Frighteners, The (1996)

예측한 평점의 영화이름. 즉, itemId가 123인 영화는 **"Frighteners, The (1996)"**인 것을 확인할 수 있음

### 3.4 user(userId=789)가 평점을 작성한 영화의 갯수를 확인

In [12]:
val movieForUser = ratings.keyBy(_.user).lookup(789)
println("Number of recommended items(movies) to userid 789 : " + movieForUser.size)

Number of recommended items(movies) to userid 789 : 33


```scala
val ratings = rawRatings.map{ case Array(user, movie, rating) => Rating(user.toInt, movie.toInt, rating.toDouble)}
```
userId가 789인 사용자가 평점을 기록한 items(영화)의 갯수는 **33개**인 것을 확인할 수 있음

### 3.3 user(userId=789)가 작성한 영화제목과 평점  확인

In [13]:
movieForUser.sortBy(-_.rating).take(10).map(rating => (titles(rating.product), rating.rating)).foreach(println)

(Godfather, The (1972),5.0)
(Trainspotting (1996),5.0)
(Dead Man Walking (1995),5.0)
(Star Wars (1977),5.0)
(Swingers (1996),5.0)
(Leaving Las Vegas (1995),5.0)
(Bound (1996),5.0)
(Fargo (1996),5.0)
(Last Supper, The (1995),5.0)
(Private Parts (1997),4.0)


### 3.4 user(userId=789)에게 추천되는 최상위 items(영화)를 확인

In [14]:
topKRecs.map(rating => (titles(rating.product), rating.rating)).foreach(println)

(Usual Suspects, The (1995),5.913209374433712)
(Bridge on the River Kwai, The (1957),5.855880700816102)
(Ben-Hur (1959),5.825030838268482)
(GoodFellas (1990),5.75696475422482)
(Terminator 2: Judgment Day (1991),5.6864516894478525)
(Fifth Element, The (1997),5.684725442684111)
(Die Hard (1988),5.6774028331597926)
(Alien (1979),5.621820401359451)
(Pulp Fiction (1994),5.614443663294212)
(Nightmare on Elm Street, A (1984),5.610997103454043)


### 사용자 추천(Cosine Similarity)

- model.userFeatures로 접근하면 됨
- model.recommendProduct()로 접근

## 4. 제품 추천(items(영화))
제품 추천은 특정 제품에 대해 그것과 가장 유사한 제품들은 어떤 것이 있는가라는 질문에 대답하는 접근방법. 

- 유사성은 설계된 모델에 종속적임. 
- 대부분의 경우 유사도는 두 제픔의 벡터 표현을 비교하여 계산됨. 

### tip) 유사도 방법론 3가지
- **피어슨-상관계수(Pearson correlation)**
- **코사인 유사도(Cosine Similarity)**
- **자커드 유사도(Jaccard Similarity)**

MatrixFactorizationModel객체는 코사인 유사도를 계산할 수 있는 API를 제공하지 않음.
새로운 함수로 구현 
- 수식 참고 : https://en.wikipedia.org/wiki/Cosine_similarity
- jblas.DoubleMatrix 이용

In [15]:
import org.jblas.DoubleMatrix // /opt/java/jre/lib/ext/jblas-1.2.3.jar

In [16]:
val aMatrix = new DoubleMatrix(Array(1.0, 2.0, 3.0))
aMatrix

[1.000000; 2.000000; 3.000000]

aMatrix는 array[Double] 자료구조를 갖음. 

### 4.1 CosineSimilarity 함수 선언

In [17]:
// 1 : 완전히 같음, 0 : 독립적(유사성X), -1 : 완전히 다름
def cosineSimilarity(vec1: DoubleMatrix, vec2: DoubleMatrix): Double = {
    vec1.dot(vec2) / (vec1.norm2() * vec2.norm2()) // norm2 : L2 regularization을 기억
}

제품 567을 위해 제품 특징 중 한 가지를 위 메서드에 적용. 모델에서 하나의 제품 특징을 lookup메서드를 통해서 수집. 

### 4.2 동일한 item의 유사성을 확인

In [18]:
val itemId = 567
// val itemId2 = 789

// 유의점 : 벡터의 크기는 중요하지 않음. 방향성만이 의미를 갖음
val itemFactor = model.productFeatures.lookup(itemId).head
// println(itemFactor.size) // itemFactor는 메모리의 주소를 참조하고 있음
val itemVector = new DoubleMatrix(itemFactor) // itemId를 사용하여 접근함
cosineSimilarity(itemVector, itemVector)

1.0

### 4.3 유사성 행렬을 items에 적용
- ItemId가 567인 영화를 기준으로 유사도를 계산

In [19]:
val sims = model.productFeatures.map{ case (id, factor) => 
    val factorVector = new DoubleMatrix(factor)
    val sim = cosineSimilarity(factorVector, itemVector)
    (id, sim)
}

In [20]:
sims.take(2)

Array((8,0.4278079357557495), (16,0.5446019074147214))

### 4.4 유사성이 높은 items(영화)를 기준으로 sorting
- top(K) 메서드는 분산환경에서 효과적으로 사용할 수 있는 메서드. collect() 대신 활용
- Ordering.by[(Int, Double), Double]{ case }
- scala doc : https://www.scala-lang.org/api/2.12.3/scala/math/Ordering.html

```scala
import scala.util.Sorting
val pairs = Array(("a", 5, 2), ("c", 3, 1), ("b", 1, 3))

// sort by 2nd element
Sorting.quickSort(pairs)(Ordering.by[(String, Int, Int), Int](_._2))
pairs // Array((b,1,3), (c,3,1), (a,5,2))

// sort by the 3rd element, then 1st
Sorting.quickSort(pairs)(Ordering[(Int, String)].on(x => (x._3, x._1)))
pairs // Array((c,3,1), (a,5,2), (b,1,3))
```

In [21]:
val sortedSims = sims.top(K)(Ordering.by[(Int, Double), Double] { case (id, similarity) => similarity})
sortedSims.take(2)

Array((567,1.0), (685,0.7458508072982897))

In [22]:
println(sortedSims.mkString("\n"))

(567,1.0)
(685,0.7458508072982897)
(1376,0.7207411742876755)
(563,0.7196819637839887)
(24,0.714372755967921)
(853,0.7035580135395358)
(719,0.7020805509981137)
(1083,0.7001007967139781)
(1178,0.6896174170300631)
(413,0.687434407639588)


결과를 확인하면 567번이 처음에 나타나는 것을 확인할 수 있음. 제거!

### 4.5 유사 제품 검사 : 영화

In [23]:
println(titles(itemId))

Wes Craven's New Nightmare (1994)


In [24]:
val sortedSims2 = sims.top(K + 1)(Ordering.by[(Int, Double), Double] { case (id, sim) => sim})
sortedSims2.slice(0, 11).map{ case (id, sim) => (titles(id), sim)}.mkString("\n")

(Wes Craven's New Nightmare (1994),1.0)
(Executive Decision (1996),0.7458508072982897)
(Meet Wally Sparks (1997),0.7207411742876755)
(Stephen King's The Langoliers (1995),0.7196819637839887)
(Rumble in the Bronx (1995),0.714372755967921)
(Braindead (1992),0.7035580135395358)
(Canadian Bacon (1994),0.7020805509981137)
(Albino Alligator (1996),0.7001007967139781)
(Major Payne (1994),0.6896174170300631)
(Tales from the Crypt Presents: Bordello of Blood (1996),0.687434407639588)
(Nightmare on Elm Street, A (1984),0.686452783648566)

## 5. 추천 모델 성능 평가
## 5.1 MSE
user-item 평점 행렬의 재구성 오차를 직접적으로 측정하는 방법. 또한 ALS를 포함해서 많은 행렬-인수분해 기술에 특화된 특정 모델에서 간소화된 객관적인 방법임. 이런 의미로 볼 때 MSE 명시적인 평가 설정 방법론으로 볼 수 있음
- 특정 사용자-제품 상에 대한 예측 평점과 실제 평점 사이 제곱 값의 차이

### 5.1.1 MSE 적용 방법 예시

In [25]:
// movieForUser는 userId = 789가 items(영화)의 평점을 작성한 결과를 갖고 있는 행렬
movieForUser.take(1)

WrappedArray(Rating(789,1012,4.0))

In [26]:
val actualRating = movieForUser.take(1)(0)
actualRating

Rating(789,1012,4.0)

사용자-제품 조합에 평점이 4.0인 것을 확인할 수 있음

In [27]:
val predictedRating = model.predict(789, actualRating.product)
predictedRating

3.9337607859393704

In [28]:
val squaredError = math.pow(predictedRating - actualRating.rating, 2.0)
squaredError

0.00438763347936991

### 5.1.2 실제 모형에 MSE 적용
1. ratings RDD에서 실제 label값을 제거하고 모형을 통해서 평점을 예측하는 모형을 설계
2. ratings RDD와 prediction RDD를 join하여 새로운 RDD를 생성(tip : 튜플형태가 고유한 키)
3. MSE적용

In [29]:
val userProducts = ratings.map{ case Rating(user, product, rating) => (user, product)}
userProducts.take(2) // (user, product)의 튜플구조를 갖음

Array((196,242), (186,302))

In [30]:
val predictions = model.predict(userProducts).map{
    case Rating(user, product, rating) => ((user, product), rating)
}
predictions.take(2)

Array(((504,384),1.8072127015030335), ((648,384),3.799487080492735))

실제 평점들을 추출해서 ratings RDD에 매핑. 동일한 형태의 키로 구성된 두 개의 RDD를 결합해서 각 사용자-제품 조합에 대한 실제 평점과 예측 평점으로 구성된 새로운 RDD를 생성

tip으로 기억해야할 점은 조인을 위해 식별을 위한 키로 튜플 형태를 사용함

In [31]:
val ratingsAndPredictions = ratings.map{
    case Rating(user, product, rating) => ((user, product), rating)
}.join(predictions)
ratingsAndPredictions.take(2)

Array(((92,386),(3.0,2.8324790803581616)), ((533,919),(2.0,1.9672505083453138)))

In [32]:
val MSE = ratingsAndPredictions.map{
    case ((user, product), (actual, predicted)) => math.pow((actual - predicted), 2)
}.reduce(_ + _) / ratingsAndPredictions.count
println("Mean Squared Error = " + MSE)

Mean Squared Error = 0.08480751044854719


### 5.1.3 RMSE(Root Mean Squared Error) 확인

In [33]:
val RMSE = math.sqrt(MSE)
println("Root Mean Squared Error = " + RMSE)

Root Mean Squared Error = 0.2912172907788052


### 5.1.4 MSE,RMSE(mllib.evaluation)

In [34]:
import org.apache.spark.mllib.evaluation.RegressionMetrics
val predictedAndTrue = ratingsAndPredictions.map{ case ((user, product), (actual, predicted)) => (actual, predicted)}
val regressionMetrics = new RegressionMetrics(predictedAndTrue)
println("Mean Squared Error = " + regressionMetrics.meanSquaredError)
println("Root Mean Squared Error = " + regressionMetrics.rootMeanSquaredError)

Mean Squared Error = 0.08480751044854719
Root Mean Squared Error = 0.2912172907788052


## 5.2 MAPK(Mean Average Precision at K)

참고 : https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision

APK는 쿼리에 대한 응답으로 나타나는 top-K 도큐먼트의 집합을 **평균 연관성 점수(average relevence score)**로 측정하는 방법. 각 쿼리 인스턴스를 사용해서 top-K 결과 집합을 실제 연관성이 있는 도큐먼트의 집합과 비교(쿼리에 대한 연관 도큐먼트의 실측 자료를 의미함)

APK 지표를 이용할 때 결과 도큐먼트는 둘 다 연관성이 있고, 연관된 두 문서가 결과 집합에서 좀 더 높은 점수가 나오면 APK 점수가 더 높게 나올 수 있기 때문에 **결과 잡합의 순서는 매우 중요**. 

이 경우 연관성이 있는 도큐먼트(실제자료)는 사용자가 요청해서 응답받는 제품의 집합을 말함. 따라서 APK는 모델을 통해 사용자가 연관성이 있는 제품을 찾고 선택할 수 있는 모든 제품을 얼마나 잘 예측하는지 측정함

### 5.2.1 APK(Average Precision K) 계산 메서드

In [35]:
def avgPrecisionK(actual: Seq[Int], predicted: Seq[Int], k: Int): Double = {
    val predK = predicted.take(k)
    var score = 0.0
    var numHits = 0.0
    for((p, i) <- predK.zipWithIndex){
        if(actual.contains(p)){
            numHits += 1.0
            score += numHits / (i.toDouble + 1.0)
        }
    }
    if (actual.isEmpty){
        1.0
    } else {
        score / scala.math.min(actual.size, k).toDouble
    }
}

avgPrecisionK 메서드는 사용자와 predicted id의 다른 목록이 서로 관련이 있는 actual 제품 ID의 목록을 입력으로 받기 때문에 평과 결과는 사용자와 연관이 있을 것임

### 5.2.1 MAPK 적용 예시

In [36]:
val actualMovies = movieForUser.map(_.product)
actualMovies.take(2) // Seq[Int] = ArrayBuffer(1012, 127)

ArrayBuffer(1012, 127)

In [37]:
// val topKRecs = model.recommendProducts(userId, K)
val predictedMovie = topKRecs.map(_.product)
predictedMovie

Array(12, 199, 526, 182, 96, 250, 144, 183, 56, 219)

In [38]:
val apk10 = avgPrecisionK(actualMovies, predictedMovie, 10)
apk10

0.0

이 경우 APK 점수가 0점이기 때문에 모델이 특정 사용자(user)에게 연관성이 있는 영화를 예측하는 작업을 잘 수행하지 못한 것을 알 수 있음

모든 사용자를 위해 APK를 연산하고 전체 APK의 평균을 계산하기 위해 모든 사용자에게 데이터 집합 중에서 추천 리스트를 생성해서 제공해야 할 것임. APK 연산은 대규모의 데이터에서 상당히 집중적일 수 있지만, 스파크 기능을 이용해서 APK 연산을 분배할 수 있음. 그렇지만 각 작업자는 전체 제품 특징 행렬을 사용해서 관련성이 있는 사용자 벡터와 모든 제품 벡터의 내적 계산을 해야하는 제약 사항이 한가지 존재함.  제품의 갯수가 방대하고 제품 행렬은 단일 머신의 메모리만 사용해야 하는 상황에서는 문제가 발생할 수 있음

### 5.2.2 실제 모형에 MAPK 적용

In [39]:
// 모든 제품
val itemFactors = model.productFeatures.map{ case (userId, factor) => factor}.collect()
// println(itemFactors.size)
val itemMatrix = new DoubleMatrix(itemFactors)
println(itemMatrix.rows, itemMatrix.columns)

(1682,50)


50개의 특징 공간에 1682개의 items로 이뤄진 행렬인 것을 확인할 수 있음

In [40]:
val imBroadcast = sc.broadcast(itemMatrix)

행렬 곱의 결과는 각 영화에 대한 예측 평점을 포함하는 벡터(길이 1682의 영화 갯수)임. 이제 예측 평점을 기준으로 모든 예측을 정렬할 것임

In [41]:
val allRecs = model.userFeatures.map{ case (userId, array) => 
    val userVector = new DoubleMatrix(array)
    val scores = imBroadcast.value.mmul(userVector)
    val sortedWithId = scores.data.zipWithIndex.sortBy(-_._1)
    val recommendedIds = sortedWithId.map(_._2 + 1).toSeq
    (userId, recommendedIds)
} // Array[(Int, Seq[Int])] = Array((8,WrappedArray(1405, 8, 1345, 25, 753, ..)))

각 사용자 ID마다 영화 ID목록을 포함하는 RDD가 생성됨. 영화 ID는 평점 순서대로 정렬되어 있음

APK 메서드에 actual 매개변수로 전달하기 위해 각 사용자에게 할당된 영화 ID의 목록도 필요함. 다행히 이미 ratings RDD가 준비되어 있으므로 여기에서 사용자와 영화 ID만 추출할 수 있음

In [42]:
val userMovies = ratings.map{ case Rating(user, product, rating) => (user, product)}.groupBy(_._1)
userMovies.take(1)

Array((778,CompactBuffer((778,94), (778,78), (778,7), (778,1273), (778,265), (778,239), (778,193), (778,1035), (778,616), (778,230), (778,582), (778,262), (778,238), (778,780), (778,195), (778,209), (778,121), (778,738), (778,204), (778,496), (778,174), (778,197), (778,161), (778,234), (778,98), (778,180), (778,56), (778,11), (778,268), (778,568), (778,367), (778,451), (778,42), (778,186), (778,150), (778,157), (778,200), (778,281), (778,54), (778,28), (778,132), (778,8), (778,144), (778,79), (778,249), (778,82), (778,755), (778,69), (778,117), (778,154), (778,143), (778,712), (778,550), (778,35), (778,246), (778,405), (778,216), (778,226), (778,219), (778,423), (778,168), (778,196), (778,441), (778,623), (778,629))))

In [43]:
val K = 10
val MAPK = allRecs.join(userMovies).map{case (userId, (predicted, actualWithIds)) => 
    val actual = actualWithIds.map(_._2).toSeq
    avgPrecisionK(actual, predicted, K)
}.reduce(_ + _) / allRecs.count
println("Mean Average Precision at K = " + MAPK)

Mean Average Precision at K = 0.03063820296588057


결과를 확인하면 낮은 APK의 평균값을 출력함. 추천 태스크에 대한 전형적인 값은 낮고, 특히 제품 집합이 대규모일 때 더욱 낮음

### 5.2.3 MAP MAPK (mllib.evaluation)
mlllib.evaluation.RankingMetrics

RankingMetrics클래스에 있는 K평균 정확도를 구현한 메서드는 앞서 구현한 것과 약간의 차이가 존재함. 그렇지만 K값을 매우 높게 선택(items의 모든 갯수)하면 대략적인 평균 정확도(MAP)의 평균 계산 값은 같음

### MAP

In [44]:
import org.apache.spark.mllib.evaluation.RankingMetrics


val predictedAndTrueForRanking = allRecs.join(userMovies).map{ case (userId, (predicted, actualWithIds)) => 
    val actual = actualWithIds.map(_._2)
    (predicted.toArray, actual.toArray)
}
val rankingMetrics = new RankingMetrics(predictedAndTrueForRanking)
println("Mean Average Precision = " + rankingMetrics.meanAveragePrecision)

Mean Average Precision = 0.07249666405791401


### MAPK

In [45]:
val MAPK2000 = allRecs.join(userMovies).map{ case (userId, (predicted, actualWithIds)) =>
    val actual = actualWithIds.map(_._2).toSeq
    avgPrecisionK(actual, predicted, 2000)
}.reduce(_ + _) / allRecs.count
println("Mean Average Precision = " + MAPK2000)

Mean Average Precision = 0.07249666405791405


### pakage vs UDF

In [46]:
val K = 2000
val MAPK = allRecs.join(userMovies).map{case (userId, (predicted, actualWithIds)) => 
    val actual = actualWithIds.map(_._2).toSeq
    avgPrecisionK(actual, predicted, K)
}.reduce(_ + _) / allRecs.count
println("Mean Average Precision at K = " + MAPK)

Mean Average Precision at K = 0.07249666405791405
